# Documentation: Topology

The topology of all elements implemented on NeoPZ are grouped in pztopology. The classes associated to this namespace are listed in the following:

 
1. TPZPoint
2. TPZLine
3. TPZQuadrilateral
4. TPZTriangle
5. TPZCube
6. TPZPrism
7. TPZPyramid
8. TPZTetrahedron

## Main atributions of the class Topology

a. Definition of number of corners, sides, dimension and faces.

In [1]:
// Make sure to change '/usr/local/' according to the installation in your computer
#pragma cling add_include_path("/usr/local/pzlib/include")
#pragma cling add_include_path("/usr/local/pzlib/include/Topology")
#pragma cling add_include_path("/usr/local/pzlib/include/Matrix")
#pragma cling add_include_path("/usr/local/pzlib/include/Util")
#pragma cling add_include_path("/usr/local/pzlib/include/Common")
#pragma cling add_include_path("/usr/local/pzlib/include/Save")
#pragma cling add_include_path("/usr/local/pzlib/include/Integral")
#pragma cling add_include_path("/usr/local/pzlib/include/LinearSolver")
#pragma cling add_include_path("/usr/local/pzlib/inclu}de/PerfUtil")

#pragma cling add_library_path("/usr/local/lib/")

#pragma cling load("libpz.so")


#include <iostream>

#include "pzvec.h"
//#include "tpzintpoints.h"
//#include "pzquad.h"
#include "tpzquadrilateral.h"

TPZVec<REAL> blob;
blob.resize(3);
blob[0] = 10.;
std::cout << blob[0] << std::endl;

// Link error to be solved:
pztopology::TPZQuadrilateral quad;
std::cout << quad.NCornerNodes << std::endl;
std::cout << quad.NSides << std::endl;
std::cout << quad.Dimension << std::endl;
std::cout << quad.NFaces << std::endl;

b. Definition of the dimension of each side and associated corner nodes.

In [2]:
# int side = 2;
# dimside = quad.SideDimension(side);
# std::cout << "The dimension of the side " << side << " is" << dimside << std::endl;
# nsidenodes = quad.NSideNodes(side);
# std::cout << "The number of associated corner nodes of the side " << side << " is" << nsidenodes << std::endl;

c. Definition of the parametric transformation between sides (note that this function returns a TPZTransform).

In [3]:
# int sidefrom = 2;
# int sideto = 1;
# TPZTransform<> tr = quad.SideToSideTransform(sidefrom, sideto);

d. Creation of integration rules associated to each side (note that the integration rule is a pointer to the TPZIntPoints type).

In [4]:
# int integr_order = 3;
# TPZIntPoints * integr = quad.CreateSideIntegrationRule(side, integr_order)

e. Definition of a transformation index associated with a side.

In [5]:
# TPZVec<int64_t> id;
# int transf_id = quad.GetTransformId(id);

f. Definition of the permutation index associated with the element.

In [6]:
# TPZVec<int> permgather; // output variable
# quad.GetSideHDivPermutation(transf_id, permgather);

g. Relationship between sides: which sides are included in the closure of a side and orientation (note that the output variable is a stack object).

In [7]:
# TPZStack<int> high;
# quad.HigherDimensionSides(side, high);

h. Projection of a point to a side (this function is implemented on TPZShape).

In [8]:
# int rib;
# TPZVec<REAL> in;
# REAL out;
# pzshape::TPZShapeQuad::ProjectPoint2dQuadToRib(rib, in, out);